<a href="https://colab.research.google.com/github/rogeriodelphi/trader_esportivo_analises/blob/main/FutPythonTrader_An%C3%A1lises_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importando as Bibliotecas
import pandas as pd
import numpy as np
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [ ]:
# Importando a Base de Dados
# df = pd.read_excel('Base_de_Dados_Full.xlsx')
df = pd.read_excel('https://github.com/rogeriodelphi/trader_esportivo_analises/blob/main/Base_de_Dados_Full_v2.xlsx?raw=true')

In [ ]:
# Renomeando as Colunas e Ordenando por Data os Jogos
df = df[['League','Date','Home','Away','FT_Odds_H','FT_Odds_D','FT_Odds_A','FT_Goals_H','FT_Goals_A']]
df.columns = ['League','Date','Home','Away','Odds_H','Odds_D','Odds_A','Goals_H','Goals_A']
df.loc[(df['Goals_H']  > df['Goals_A']), "Result"] = "H"
df.loc[(df['Goals_H'] == df['Goals_A']), "Result"] = "D"
df.loc[(df['Goals_H'] <  df['Goals_A']), "Result"] = "A"

df['Date'] = df['Date'].dt.strftime('%Y/%m/%d')
df = df.sort_values('Date')

In [ ]:
# Ajustando o Índice
df.reset_index(inplace=True, drop=True)
df.index = df.index.set_names(['Nº do Jogo'])
df = df.rename(index=lambda x: x + 1)

In [ ]:
# Visualizando o DataFrame
df

In [ ]:
# Criando a Coluna de Pontos Obtidos
def Pontos_H (Result):
    if Result == 'H':
        Pontos = 3
    elif Result == 'D':
        Pontos = 1
    else:
        Pontos = 0
    return Pontos

def Pontos_A (Result):
    if Result == 'A':
        Pontos = 3
    elif Result == 'D':
        Pontos = 1
    else:
        Pontos = 0
    return Pontos

df['Pontos_H'] = df.apply(lambda row: Pontos_H(row['Result']), axis=1)
df['Pontos_A'] = df.apply(lambda row: Pontos_A(row['Result']), axis=1)

In [ ]:
# Visualizando o DataFrame
df

In [ ]:
df['Total_Goals'] = df.apply(lambda row: (row['Goals_H'] + row['Goals_A']), axis=1)

In [ ]:
df['Over05'] = df.apply(lambda row: 1 if (row['Total_Goals'] >= 1) else 0, axis=1)
df['Over15'] = df.apply(lambda row: 1 if (row['Total_Goals'] > 1) else 0, axis=1)
df['Over25'] = df.apply(lambda row: 1 if (row['Total_Goals'] > 2) else 0, axis=1)
df['BTTS'] = df.apply(lambda row: 1 if (row['Goals_H'] > 0 and row['Goals_A'] > 0) else 0, axis=1)

In [ ]:
df.head()

In [ ]:
media_over05_H = df[['Home','Over05']].groupby('Home').mean() * 100
media_over15_H = df[['Home','Over15']].groupby('Home').mean() * 100
media_over25_H = df[['Home','Over25']].groupby('Home').mean() * 100
media_BTTS_H = df[['Home','BTTS']].groupby('Home').mean() * 100

media_over05_A = df[['Away','Over05']].groupby('Away').mean() * 100
media_over15_A = df[['Away','Over15']].groupby('Away').mean() * 100
media_over25_A = df[['Away','Over25']].groupby('Away').mean() * 100
media_BTTS_A = df[['Away','BTTS']].groupby('Away').mean() * 100

In [ ]:
df_medias_H=pd.concat([media_over05_H,media_over15_H,media_over25_H,media_BTTS_H],axis=1)

df_medias_A=pd.concat([media_over05_A,media_over15_A,media_over25_A,media_BTTS_A],axis=1)

In [ ]:
df_medias_H.columns = ['Média_Over05_H','Média_Over15_H','Média_Over25_H','Média_BTTS_H']
df_medias_H.to_excel("medias_H.xlsx")
df_medias_A.columns = ['Média_Over05_A','Média_Over15_A','Média_Over25_A','Média_BTTS_A']
df_medias_A.to_excel("medias_A.xlsx")

In [ ]:
df_medias_H

In [ ]:
df_medias_A

In [ ]:
# Analisando dois times específicos
home = input("Digite o nome do Mandante: ")
away = input("Digite o nome do Visitante: ")

flt1 = df_medias_H[df_medias_H.index == home];
flt2 = df_medias_A[df_medias_A.index == away];

display(flt1, flt2)

In [ ]:
# Calculando as Médias de Gols, Desvio Padrão e Coeficiente de Variação
media_gols_home = round(df[['Home', 'Goals_H']].groupby('Home').mean(), 2)
media_gols_away = round(df[['Away','Goals_A']].groupby('Away').mean(), 2)

desvio_padrao_gols_home = round(df[['Home', 'Goals_H']].groupby('Home').std(), 2)
desvio_padrao_gols_away = round(df[['Away', 'Goals_A']].groupby('Away').std(), 2)

cv_gols_home = round(desvio_padrao_gols_home / media_gols_home, 2)
cv_gols_away = round(desvio_padrao_gols_away / media_gols_away, 2)

df_goals=pd.concat([media_gols_home,media_gols_away,desvio_padrao_gols_home,desvio_padrao_gols_away,cv_gols_home,cv_gols_away],axis=1)

In [ ]:
# Calculando as Médias de Pontos, Desvio Padrão e Coeficiente de Variação
media_pontos_home = round(df[['Home','Pontos_H']].groupby('Home').mean(), 2)
media_pontos_away = round(df[['Away','Pontos_A']].groupby('Away').mean(), 2)

desvio_padrao_pontos_home = round(df[['Home','Pontos_H']].groupby('Home').std(), 2)
desvio_padrao_pontos_away = round(df[['Away','Pontos_A']].groupby('Away').std(), 2)

cv_pontos_home = round(desvio_padrao_pontos_home / media_pontos_home, 2)
cv_pontos_away = round(desvio_padrao_pontos_away / media_pontos_away, 2)

df_pontos=pd.concat([media_pontos_home,media_pontos_away,desvio_padrao_pontos_home,desvio_padrao_pontos_away,cv_pontos_home,cv_pontos_away],axis=1)

In [ ]:
df_analise = pd.concat([df_goals,df_pontos],axis=1)
df_analise.columns = ['Média_Gols_Home','Média_Gols_Away','DP_Gols_Home','DP_Gols_Away','CV_Gols_Home','CV_Gols_Away','Média_Pontos_Home','Média_Pontos_Away','DP_Pontos_Home','DP_Pontos_Away','CV_Pontos_Home','CV_Pontos_Away']
df_analise = df_analise[['Média_Gols_Home','Média_Gols_Away','CV_Gols_Home','CV_Gols_Away','Média_Pontos_Home','Média_Pontos_Away','CV_Pontos_Home','CV_Pontos_Away']]
df_analise = df_analise.dropna()
df_analise.to_excel("analise.xlsx")

In [ ]:
# Exibindo o DataFrame
df_analise

In [ ]:
# Analisando dois times específicos
home = input("Digite o nome do Mandante: ")
away = input("Digite o nome do Visitante: ")

flt1 = df_analise[df_analise.index == home];
flt2 = df_analise[df_analise.index == away];

display(flt1, flt2)